CHOOSE THE ATTRIBUTE

In [ ]:
#attr = "age"
attr = "star"
#attr = "eth"
#attr = "gender"

In [ ]:
import pandas as pd
from fairness_all_functions import * 
import random
from ast import literal_eval
from pathlib import Path
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt
from collections import Counter


# Read the CSV file into a DataFrame
df_reviews = pd.read_csv("dataset/yelp_reviews.csv")
df_items = pd.read_csv("dataset/yelp_items_with_rating.csv")

list_items = df_items["item_id"].tolist()
num_items = len(list_items)
print(num_items)

df_reviews['ethnicity'] = df_reviews['ethnicity'].astype(str)
df_reviews['age'] = df_reviews['age'].fillna(-1).astype(int)


# Add utility column with 0 as minimum

df_reviews['utility'] = None 
for i in range(len(df_reviews)):
    utility_i = eval(df_reviews.iloc[i].feedback)["counts"]["useful"]
    df_reviews.loc[i,"utility"] = utility_i +1

# Function to map age to age group
def age_to_group(age):
    if age < 0:
        return 'nan'
    elif age <= 25:
        return '0-25'
    elif age <= 45:
        return '26-45'
    elif age <= 65:
        return '46-65'
    else:
        return 'over 65'

# Apply the function to the 'age' column
df_reviews['age_group'] = df_reviews['age'].apply(age_to_group)

# utility function to parse Lorenzo's data (funzione di Jerin!)
def parse_into_dataframe(text_filepath: Path):

    with open(text_filepath, 'r') as file:
        data = file.read()

    data = data.split('\n')
    
    parsed_data = []
    for line in tqdm(data):
        if line:
            parsed_data.append(literal_eval(line))
    
    # create a dataframe
    df = pd.DataFrame(parsed_data, columns=column_names)
    return df

df_reviews['gender'] = df_reviews['gender'].astype(str)
#df_reviews["gender"].head(10)

In [ ]:
print(len(df_reviews))

In [ ]:
# number_reviews is the number of reviews per restaurant
number_reviews = 10000
number_kendall = 10 # BETTER 100

In [ ]:
if attr == "age":
    list_attr = list(set(df_reviews['age_group']))
    list_attr = ['nan','26-45','0-25','46-65','over 65']
    list_symbols = ['nan','26-45','0-25','46-65','over 65']
    column_str = 'age_group'

if attr == "gender":
    list_attr = list(set(df_reviews['age_group']))
    list_attr = ['nan','feminine','masculine']
    list_symbols = ['nan','feminine','masculine']
    column_str = 'gender'

if attr == "star":
    list_attr = list(set(df_reviews['rating']))
    list_attr = [5,4,3,2,1]
    list_symbols =['5','4','3','2','1']
    column_str = 'rating'
if attr == "eth":
    list_attr = list(set(df_reviews['ethnicity']))
    #list_attr.remove('american indian or alaska native')
    list_attr = ['nan', 'white', 'asian', 'hispanic, latino, or spanish origin', 'black or african american', 'middle eastern or north african', 
            'native hawaiian or pacific islander', 'american indian or alaska native']
    list_symbols = ['nan','wh','as','hi','bl','mi','ha','am']
    column_str = 'ethnicity'

column_names = list_attr
num_attr = len(list_attr)
print(list_attr)

REMOVE NAN

In [ ]:
list_attr = [s for s in list_attr if s != "nan"]
list_symbols = [s for s in list_symbols if s != "nan"]
column_names = list_attr
num_attr = len(list_attr)

In [ ]:
# For every value in list_attr, count the molteplicity for each item

attr_dict = {key: None for key in list_attr}

for attr_i in list_attr:
    multiplicity_i = 0
    for item in list_items:
        x = (df_reviews[column_str][df_reviews["item_id"] == item].head(number_reviews)).tolist()
        multiplicity = Counter(x)
        #print(multiplicity)
        multiplicity_i += multiplicity[attr_i]

    attr_dict[attr_i] = multiplicity_i

attr_dict

In [ ]:
def divide_dict_values_by_sum(d):
    # Calculate the sum of all values in the dictionary
    total_sum = sum(d.values())
        
    # Divide each value in the dictionary by the total sum
    result_dict = {key: round((value / total_sum),4) for key, value in d.items()}
    
    return result_dict

# Divide all values of the dictionary by the sum of all values
frequencies = divide_dict_values_by_sum(attr_dict)

print(frequencies)


new_frequencies is used to replace "nan" values

In [ ]:
if False:
    new_frequencies = {key: None for key in frequencies.keys()}
    new_frequencies["nan"] = 0
    for key in frequencies.keys():
        if key != "nan":
            print(key)
            new_frequencies[key] = frequencies[key]/(1-frequencies["nan"])

    print(new_frequencies)

    # Extract keys and probabilities
    keys = list(new_frequencies.keys())
    weights = list(new_frequencies.values())

    for index, row in df_reviews.iterrows():
        if row["ethnicity"] == "nan":
            x = random.choices(keys, weights=weights)[0]
            df_reviews.at[index,"ethnicity"] = x
        

ONE LEVEL

We take only the first 50 for each item

In [ ]:
list_of_lists_items_exp_avg = list()
list_of_lists_items_exp_top = list()
#list_of_lists_items_treatment_avg = list()
list_of_lists_items_treatment_top = list()
list_of_lists_items_kendall = list()
for i in range(num_items):
        list_of_lists_items_exp_avg.append(list())
        list_of_lists_items_exp_top.append(list())
        #list_of_lists_items_treatment_avg.append(list())
        list_of_lists_items_treatment_top.append(list())
        list_of_lists_items_kendall.append(list())


for i in range(num_items):
        item_i = list_items[i]
        df_reviews_i = df_reviews[df_reviews["item_id"] == item_i].head(number_reviews)
        df_reviews_i_kendall = df_reviews[df_reviews["item_id"] == item_i].head(number_kendall)

        list_df_attr_i = []
        list_df_attr_i_kendall = []
        for j in range(num_attr):
                list_df_attr_i.append(df_reviews_i[df_reviews_i[column_str] == list_attr[j]]) 
                list_df_attr_i_kendall.append(df_reviews_i_kendall[df_reviews_i_kendall[column_str] == list_attr[j]])



        for j in range(num_attr):
                if len(list_df_attr_i[j]) > 0:
                        list_of_lists_items_exp_avg[i].append(exposure_avg(list_df_attr_i[j]))
                        list_of_lists_items_exp_top[i].append(exposure_top(list_df_attr_i[j]))
                        #list_of_lists_items_treatment_avg[i].append(treatment_avg(list_df_attr_i[j]))
                        list_of_lists_items_treatment_top[i].append(treatment_top(list_df_attr_i[j]))
                else:
                        #print("HO UN CASO NULLO", list_attr[j])
                        list_of_lists_items_exp_avg[i].append(0)
                        list_of_lists_items_exp_top[i].append(0)
                        #list_of_lists_items_treatment_avg[i].append(0)
                        list_of_lists_items_treatment_top[i].append(0)


                if len(list_df_attr_i_kendall[j]) > 0:
                        # PER LA KENDALL DEVO CALCOLARE IL DF DIFFERENCE
                        merged_df = df_reviews_i_kendall.merge(list_df_attr_i_kendall[j], how='left', indicator=True)
                        difference_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')

                        if (len(difference_df) != 0):
                                list_of_lists_items_kendall[i].append(kendall_tau_asymmetric(list_df_attr_i_kendall[j], difference_df))
                else:
                        list_of_lists_items_kendall[i].append(-2)


                

In [ ]:
with open('data_boxplot/one_level/list_of_lists_items_exp_avg.txt', 'w') as file:
    for sublist in list_of_lists_items_exp_avg:
        file.write('[' + ', '.join(map(str, sublist)) + ']\n')

with open('data_boxplot/one_level/list_of_lists_items_exp_top.txt', 'w') as file:
    for sublist in list_of_lists_items_exp_top:
        file.write('[' + ', '.join(map(str, sublist)) + ']\n')

#with open('data_boxplot/one_level/list_of_lists_items_treatment_avg.txt', 'w') as file:
#    for sublist in list_of_lists_items_treatment_avg:
#        file.write('[' + ', '.join(map(str, sublist)) + ']\n')

with open('data_boxplot/one_level/list_of_lists_items_treatment_top.txt', 'w') as file:
    for sublist in list_of_lists_items_treatment_top:
        file.write('[' + ', '.join(map(str, sublist)) + ']\n')

with open('data_boxplot/one_level/list_of_lists_items_kendall.txt', 'w') as file:
    for sublist in list_of_lists_items_kendall:
        file.write('[' + ', '.join(map(str, sublist)) + ']\n')


In [ ]:
data_dirpath = Path.cwd()/ 'data_boxplot/one_level'
# list the text files in the data directory
text_files = list(data_dirpath.glob('*.txt'))

# iterate through the text files and parse the data
fairness_measure_to_df = {}
for filepath in text_files:
    filename = filepath.stem
    filename = filename.split('_')
    fairness_measure_name = filename[-2:]
    fairness_measure_name = '_'.join(fairness_measure_name)
    fairness_measure_to_df[fairness_measure_name] = parse_into_dataframe(filepath)

# replace every -2 with NaN for the item_kendall fairness measure and
# every 0 with NaN for the other fairness measures
for fairness_measure, df in fairness_measure_to_df.items():
    if fairness_measure == 'items_kendall':
        fairness_measure_to_df[fairness_measure] = df.replace(-2, pd.NA)
    else:
        fairness_measure_to_df[fairness_measure] = df.replace(0, pd.NA)

# generate a boxplot for each fairness measure
# and save the plot as an html file
for fairness_measure, df in fairness_measure_to_df.items():

    print(fairness_measure)
    fig = go.Figure()
    for column in df.columns:
        fig.add_trace(go.Box(y=df[column], name=column))
    
    # set the y-axis range to be between 0 and 1 except for the item_kendall fairness measure
    # where the range is between -1 and 1
    if fairness_measure == 'items_kendall':
        fig.update_layout(yaxis_range=[-1.1, 1.1])
        #fig.update_layout(title = fairness_measure)
    else:
        fig.update_layout(yaxis_range=[-0.1, 1.1])
        #fig.update_layout(title =  str(fairness_measure)+str(" for ") + str(column_str) + str(" with first ")+ str(number_reviews)+ " reviews")

    
    #fig.update_layout(showlegend=False,xaxis=dict(title='', showticklabels=False), width=300, height=250)
    fig.update_layout(showlegend=False, width=300, height=250)

    # Define the custom labels you want to use
    fig.update_xaxes(tickvals=list_attr, ticktext=list_symbols)

    #fig.update_layout(title = fairness_measure)

    #if fairness_measure == 'items_kendall':
    #    fig.update_layout(annotations=[dict(x=-0.12, y=0.5, text="rank_equality", showarrow=False, textangle=-90, 
    #                      xref="paper", yref="paper", font=dict(size=14))])      
    #else:
    #    fig.update_layout(annotations=[dict(x=-0.12, y=0.5, text=fairness_measure, showarrow=False, textangle=-90, 
    #                      xref="paper", yref="paper", font=dict(size=14))])
    fig.update_layout(margin=dict(l=5, r=5, t=5, b=0))
    #fig.update_layout(showlegend=True)

    #fig.update_layout(
    #xaxis_title=['5','4','3','2','1']
    #)
    
    fig.write_image("data_boxplot/plot/one_level/"+str(fairness_measure)+".png")
    fig.show()


    



TWO LEVELS

In [ ]:
df_query1 = pd.read_csv("query_yelp/query1.csv")
df_query2 = pd.read_csv("query_yelp/query2.csv")
df_query3 = pd.read_csv("query_yelp/query3.csv")
df_query4 = pd.read_csv("query_yelp/query4.csv")
df_query5 = pd.read_csv("query_yelp/query5.csv")
df_query6 = pd.read_csv("query_yelp/query6.csv")

df_query1 = df_query1.drop("query_id", axis=1)
df_query2 = df_query2.drop("query_id", axis=1)
df_query3 = df_query3.drop("query_id", axis=1)
df_query4 = df_query4.drop("query_id", axis=1)
df_query5 = df_query5.drop("query_id", axis=1)
df_query6 = df_query6.drop("query_id", axis=1)

In [ ]:
#list(df_query["ordering_attributes"])

In [ ]:
#df_query = df_query1 # OR
#df_query = df_query2 # OR
#df_query = df_query3 # OR
df_query = df_query4 # OR
#df_query = df_query5 # OR
#df_query = df_query6 

df_query["query_attributes"] = df_query["query_attributes"].apply(literal_eval)
df_query["ordering_attributes"] = df_query["ordering_attributes"].apply(literal_eval)

number_reviews_two_levels = 15

# ADD TYPE FIELD
df_query['type'] = None
for index, row in df_query.iterrows():
    #print(row["ordering_attributes"])
    if row["ordering_attributes"]:
        df_query.loc[index,'type'] = row["ordering_attributes"][0]
    else:
        df_query.loc[index,'type'] = "none"

print(set(df_query["type"]))

pd.set_option('mode.chained_assignment', None)

# DELETE "none" and fix ordering_id
df_query = df_query[df_query ["type"] != "none"]
df_query["ordering_id"] = df_query["ordering_id"] -1

k = len(df_query[df_query["ordering_id"] == 2])
p = max(df_query["ordering_id"])
print("numero di permutazioni, numero di ristoranti =",p,",",k)

In [ ]:
#list(df_query["ordering_attributes"])

In [ ]:
if True:
    # DELETE ALL ROWS SO THAT ['ordering_attributes'] IS LONGER THAN 1
    df_query = df_query[df_query['ordering_attributes'].apply(lambda x: len(x) <= 1)]

    # TRASFORM ROWS
    df_query['ordering_direction'] = df_query['ordering_direction'].apply(lambda x: eval(x))
    df_query['ordering_direction'] = df_query['ordering_direction'].apply(lambda x: x[0])

    #print(df_query['ordering_attributes'])

    #df_query['ordering_attributes'] = df_query['ordering_attributes'].apply(lambda x: eval(x))
    df_query['ordering_attributes'] = df_query['ordering_attributes'].apply(lambda x: x[0])

    df_query['string'] = df_query['ordering_attributes'] + ' ' + df_query['ordering_direction']
    #df_query

    # CHANGE SOME VALUES AND DELETE "distance asc"
    df_query['string'] = df_query['string'].replace('attributes.RestaurantsPriceRange2 desc', 'price desc')
    df_query['string'] = df_query['string'].replace('attributes.RestaurantsPriceRange2 asc', 'price asc')
    df_query['string'] = df_query['string'].replace('review_count desc', '#reviews desc')
    df_query = df_query[df_query['string'] != "distance desc"]
    df_query.reset_index(inplace=True)


    number_singles = int(len(df_query)/k)
    #display(sub_df)
    repeated_numbers = [num for num in range(1, number_singles+1) for _ in range(k)]
    print(repeated_numbers)
    print(len(repeated_numbers), len(df_query))
    df_query["ordering_id"] = repeated_numbers

    df_query.reset_index(inplace=True)

    k = len(df_query[df_query["ordering_id"] == 2])
    p = max(df_query["ordering_id"])
    print("number of permutations, number of restaurant =",p,",",k)

In [ ]:
# Dataframe 2 levels
if True:
    exp_avg_2_levels = list() 
    exp_top_2_levels = list() 
    treatment_top_2_levels = list() 
    for j in range(p):
            exp_avg_2_levels.append(list())
            exp_top_2_levels.append(list())
            treatment_top_2_levels.append(list())


    for i in range(p):

        permutation_items = df_query[df_query["ordering_id"] == i+1]["item_id"].tolist()

        list_of_lists_items_exp_avg = list()
        list_of_lists_items_exp_top = list()
        list_of_lists_items_treatment_top = list()
        for j in range(k):
                list_of_lists_items_exp_avg.append(list())
                list_of_lists_items_exp_top.append(list())
                list_of_lists_items_treatment_top.append(list())


        for z in range(len(permutation_items)):
            #print("z = ", z)

            item = permutation_items[z]
            df_i = df_reviews[df_reviews["item_id"] == item].head(number_reviews_two_levels)

            list_df_attr_i = []
            for j in range(num_attr):
                    list_df_attr_i.append(df_i[df_i[column_str] == list_attr[j]]) 


            for j in range(num_attr):
                if len(list_df_attr_i[j]) > 0:
                        list_of_lists_items_exp_avg[z].append(exposure_avg(list_df_attr_i[j]))
                        list_of_lists_items_exp_top[z].append(exposure_top(list_df_attr_i[j]))
                        list_of_lists_items_treatment_top[z].append(treatment_top(list_df_attr_i[j]))
                else:
                        list_of_lists_items_exp_avg[z].append(0)
                        list_of_lists_items_exp_top[z].append(0)
                        list_of_lists_items_treatment_top[z].append(0)

        normalization_factor = 0
        for w in range(k):
                normalization_factor += (1/(math.log(w+2,2)))


        for x in range(num_attr):
            count = 0
            for w in range(k):
                count += (1/(math.log(w+2,2))) * list_of_lists_items_exp_avg[w][x]
            exp_avg_2_levels[i].append(count/normalization_factor)

        for x in range(num_attr):
            count = 0
            for w in range(k):
                count += (1/(math.log(w+2,2))) * list_of_lists_items_exp_top[w][x]
            exp_top_2_levels[i].append(count/normalization_factor)

        for x in range(num_attr):
            count = 0
            for w in range(k):
                count += (1/(math.log(w+2,2))) * list_of_lists_items_treatment_top[w][x]
            treatment_top_2_levels[i].append(count/normalization_factor)  

# Save the dataframes
if True:
    with open('data_boxplot/two_levels/list_of_lists_items_exp_avg.txt', 'w') as file:
        for sublist in exp_avg_2_levels:
            file.write('[' + ', '.join(map(str, sublist)) + ']\n')

    with open('data_boxplot/two_levels/list_of_lists_items_exp_top.txt', 'w') as file:
        for sublist in exp_top_2_levels:
            file.write('[' + ', '.join(map(str, sublist)) + ']\n')

    with open('data_boxplot/two_levels/list_of_lists_items_treatment_top.txt', 'w') as file:
        for sublist in treatment_top_2_levels:
            file.write('[' + ', '.join(map(str, sublist)) + ']\n')

# Create the fairness_measures
if True:
    data_dirpath = Path.cwd()/ 'data_boxplot/two_levels'
    # list the text files in the data directory
    text_files = list(data_dirpath.glob('*.txt'))

    # iterate through the text files and parse the data
    fairness_measure_to_df = {}
    for filepath in text_files:
        filename = filepath.stem
        filename = filename.split('_')
        fairness_measure_name = filename[-2:]
        fairness_measure_name = '_'.join(fairness_measure_name)
        fairness_measure_to_df[fairness_measure_name] = parse_into_dataframe(filepath)

    # replace every -2 with NaN for the item_kendall fairness measure and
    # every 0 with NaN for the other fairness measures
    for fairness_measure, df in fairness_measure_to_df.items():
        if fairness_measure == 'items_kendall':
            fairness_measure_to_df[fairness_measure] = df.replace(-2, pd.NA)
        else:
            fairness_measure_to_df[fairness_measure] = df.replace(0, pd.NA)

    # generate a boxplot for each fairness measure
    # and save the plot as an html file

    # TO PLOT THE BOXPLOT SET AS TRUE

    if False:
        for fairness_measure, df in fairness_measure_to_df.items():

            print(fairness_measure)
            fig = go.Figure()
            for column in df.columns:
                fig.add_trace(go.Box(y=df[column], name=column))
            
            # set the y-axis range to be between 0 and 1 except for the item_kendall fairness measure
            # where the range is between -1 and 1
            if fairness_measure == 'items_kendall':
            #    fig.update_layout(yaxis_range=[-1, 1], title = fairness_measure)
                fig.update_layout(yaxis_range=[-1, 1])
            #if fairness_measure != 'exp_top':
            #    fig.update_layout(yaxis_range=[0, 1], title = str(fairness_measure)+str(" con ")+str(k)+" ristoranti e "+ str(number_reviews)+ " a ristorante")
                
            else:
                #fig.update_layout(yaxis_range=[0, 1], title = str(fairness_measure)+str(" con ")+str(k)+" ristoranti e "+ str(number_reviews)+ " a ristorante")
                fig.update_layout(yaxis_range=[0, 1])


            fig.update_layout(showlegend=False,xaxis=dict(title='', showticklabels=False), width=300, height=250)
            fig.update_layout(margin=dict(l=5, r=5, t=5, b=0))

            fig.write_image("data_SCATOLE_E_BAFFI/plot/two_levels/"+str(fairness_measure)+"_two_levels"+".png")
            #fig.show()
        

In [ ]:
#fig, ax = plt.subplots(2, 1)
#plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.4, hspace=0.4)
for measure in ["exp_top", "treatment_top"]:
#for measure in ["exp_top"]:
    df = fairness_measure_to_df[measure]

    sublist = (list(df_query["string"]))[::k]
    df['color'] = sublist

    column_names_map = dict(zip(list_attr, list_symbols))
    # Rename columns
    df = df.rename(columns=column_names_map)
    df = df.dropna(axis=1, how='all')

    plt.figure(figsize=(10, 4))
    plt.tight_layout()
    #plt.update_layout(margin=dict(l=5, r=5, t=5, b=0))

    pd.plotting.parallel_coordinates(df, 'color', color=("#000000","#009292","#ff6db6","#b6dbff", "#920000","#db6d00","#24ff24","#ffff6d"))
    plt.savefig('data_boxplot/two_levels/'+measure+'.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
# Create a figure with 1 row and 2 columns
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

#fig, ax = plt.subplots(2, 1)
#plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, wspace=0.4, hspace=0.4)
for (measure, ax) in zip(["exp_top", "treatment_top"], [ax1, ax2]):
#for measure in ["exp_top"]:
    df = fairness_measure_to_df[measure]

    sublist = (list(df_query["string"]))[::k]
    df['color'] = sublist

    column_names_map = dict(zip(list_attr, list_symbols))
    # Rename columns
    df = df.rename(columns=column_names_map)
    df = df.dropna(axis=1, how='all')

    
    #plt.update_layout(margin=dict(l=5, r=5, t=5, b=0))

    pd.plotting.parallel_coordinates(frame=df, ax=ax, class_column='color', color=("#000000","#009292","#ff6db6","#b6dbff", "#920000","#db6d00","#24ff24","#ffff6d"))
    #plt.savefig('data_boxplot/two_levels/'+measure+'.png', bbox_inches='tight')

plt.tight_layout()
plt.savefig('data_boxplot/two_levels/exp_treatment_top_subplot.pdf', bbox_inches='tight')
plt.show()


In [ ]:
fairness_measure_to_df["treatment_top"]